In [1]:
import pandas as pd
import random
from datetime import timedelta
from datetime import datetime
import duckdb as ddb

# This is a sample of what a product sheet would look like
# ID for vendor will mean something such as type of product or payment type
product_df = pd.DataFrame({
    "ProductID": [1,2,3,4],
    "Product": ["eggs","wine","cheese","candles"],
    "Vendor": [3001, 3002, 3003, 3004],
    "Unit_Price": [3.20, 16.20, 5.20,20.5],
    "Unit_Cost": [1.20, 3.40, 2.10, 10],
    "Initial_Inventory": [0,0,0,0],
    "Spread":[.30, .50, .10, .10],
    "PerInvoiceRng_Min":[10,10,10,10],
    "PerInvoiceRng_Max":[20,20,20,20]
})

# This would all come in as JSON not created in Python
Initial_Year = 2010
YearsIn_Operation = 10
first_year_rev = 10000000
trend = .05
modjulation = .01

x = first_year_rev
y = Initial_Year
coin = (1,2)

list_years = []



for year in range(YearsIn_Operation):
  Revenue = x
  Year = y
  Trended = Revenue * (1+trend)
  direction = random.sample(coin,1)
  if direction == 1:
    Modjulated = Trended * (1+modjulation)
  else:
    Modjulated = Trended * (1-modjulation)
  list_year_item = [Year, round(Revenue,2)]
  list_years.append(list_year_item)
  x = Modjulated
  y = y+1

# This is what the Data would look like in JSON
rev_df = pd.DataFrame(list_years,columns=["Year","Rev"])


In [2]:
# from re import S
class PreProcessing:
  def __init__(self, product_df, year, Year_Rev):
    self.revenue = Year_Rev[1]
    self.products = product_df
    self.year = year

  def ProductProcessing(self):
    self.product_id_list = self.products['ProductID']
    initial_inventory = self.products[['Product','Initial_Inventory']].to_dict(orient='records')
    self.initial_inventory_dict = {d['Product']:d['Initial_Inventory'] for d in initial_inventory}

    self.product_cost_info = self.products[['Product', 'Vendor','Spread',
                                            'Unit_Price', 'Unit_Cost',
                                            'PerInvoiceRng_Min',
                                            'PerInvoiceRng_Max']].sort_values(by='Spread', ascending=False)
    product_cost = self.product_cost_info
    Rev = self.revenue
    yr_Product_Rev = []
    yr_Quantity_Sold = []
    Product_Cost = []
    Selling_Price = []
    yr_Product = []
    yr_noRecords_min = []
    yr_noRecords_max = []
    vendors = []
    for product in self.product_cost_info['Product']:
      vendor = product_cost[product_cost['Product']==product]['Vendor']
      noRecords_min = product_cost[product_cost['Product']==product]['PerInvoiceRng_Min']
      noRecords_max = product_cost[product_cost['Product']==product]['PerInvoiceRng_Max']
      spread = product_cost[product_cost['Product']==product]['Spread'].item()
      unit_price = product_cost[product_cost['Product']==product]['Unit_Price'].item()
      unit_cost = product_cost[product_cost['Product']==product]['Unit_Cost'].item()
      price_sold = self.revenue * spread
      q_sold = round(price_sold / unit_price,0)
      p_sold = q_sold*unit_price
      if Rev >= p_sold:
        Rev = Rev - p_sold
      else:
        q_sold = round(Rev / unit_price,0)
        p_sold = q_sold*unit_price

      vendors.append(vendor)
      yr_Product.append(product)
      yr_Product_Rev.append(p_sold)
      yr_Quantity_Sold.append(q_sold)
      Product_Cost.append(unit_cost)
      Selling_Price.append(unit_price)
      yr_noRecords_min.append(noRecords_min)
      yr_noRecords_max.append(noRecords_max)

    processed_product_lists = [yr_Product, yr_Product_Rev,
                               yr_Quantity_Sold, Product_Cost, Selling_Price,
                               yr_noRecords_min, yr_noRecords_max, vendors]

    self.Processed_Product_df = pd.DataFrame(processed_product_lists).T
    self.Processed_Product_df.columns = ["yr_Product","yr_Product_Rev",
                                         "yr_Quantity_Sold","Product_Cost","Selling_Price",
                                         "yr_noRecordsMin","yr_noRecordsMax", "Vendors"]
    return [self.year, self.Processed_Product_df, self.initial_inventory_dict]

In [14]:

class CompanyGeneration:
  def __init__(self,RF, yera, product, yr_product_data):
    self.RF = RF
    self.year = yera
    self.yr_product_data = yr_product_data
    self.Sales_df = pd.DataFrame({"A":[1,2,3],"B":[4,5,6]})

  def GenerateSales(self): # Already Per Product
    print(self.year)
    yr_Quantity_Sold = yr_product_data[yr_product_data['yr_Product']==product]['yr_Quantity_Sold'].item()
    Product_Cost = yr_product_data[yr_product_data['yr_Product']==product]['Product_Cost'].item()
    Selling_Price = yr_product_data[yr_product_data['yr_Product']==product]['Selling_Price'].item()
    MinRecords_PerInvoice = int(yr_product_data[yr_product_data['yr_Product']==product]['yr_noRecordsMin'])
    MaxRecords_PerInvoice = int(yr_product_data[yr_product_data['yr_Product']==product]['yr_noRecordsMax'])
    Vendor = yr_product_data[yr_product_data['yr_Product']==product]['Vendors'].item()

    # Initial Quantity
    Quantity_Remaining = yr_Quantity_Sold
    Yr_Start = datetime(self.year, 1, 1)
    Yr_End = datetime(self.year, 12, 31)

    Sales_df_rows = []
    x = 0

    while Quantity_Remaining > 0:
      Order_Date = Yr_Start + timedelta(days=random.randint(0, (Yr_End-Yr_Start).days))
      # Product is Shipped 1-30 days after order
      Shipment_Date = Order_Date + timedelta(days = random.randint(1, 30))
      Records_PerInvoice = random.randint(MinRecords_PerInvoice, MaxRecords_PerInvoice)
      if Records_PerInvoice > Quantity_Remaining:
        Records_PerInvoice = Quantity_Remaining
      else:
        pass
      Rev = Records_PerInvoice * Selling_Price
      Tcost = Records_PerInvoice * Product_Cost
      Sales_df_rows.append([product, Records_PerInvoice, Order_Date, Shipment_Date, Selling_Price, Product_Cost, Rev, Tcost, Vendor])
      Quantity_Remaining = Quantity_Remaining - Records_PerInvoice

      # Record Journal Entries for Rev and Cash or AR

    self.Sales_df = pd.DataFrame(Sales_df_rows, columns = ["Product", "Quantity",
                                                      "Order_Date", "Shipment_Date",
                                                      "Selling_Price", "Product_Cost",
                                                      "Rev", "Total_cost", "Vendors"])

    cleaned_df = self.Sales_df[['Shipment_Date', 'Rev']].copy()
    cleaned_df.rename(columns={'Shipment_Date': 'Date', 'Rev': 'Amount'}, inplace=True)
    self.RF.JournalEntries_Bulk(cleaned_df,1000,4000,"dr",1010,1000,70,30)
    # 1000 - Cash | 1010 - Accounts Payable | 4000 - Revenue

    return [self.year, self.Sales_df]

  def GeneratePO(self, Master_Inventory):
    Sales_df = self.Sales_df
    Product_Ordered = []
    self.Quantity_Purchased = []
    self.Date_Purchased = []
    self.InventoryBeg = []
    self.InventoryEnd = []
    Vendor = []

    Steps = '''
            Steps
            1. Everything is already grouped my year and product, so just group purchases by quarter,
            then for each quarter check if the product purchased is more than the current quarterly
            inventory
            2. Do this on a rolling basis, then:
              2a. If it is more, deduct from the rolling inventory
              2b. If it is less than add 1.05% and then deduct the inventory
            3. Record all additions in the PO Journal and call the JE func.
            '''

    qry_PO = '''
      SELECT Vendors AS Vendor,
      Product,
      SUM(Quantity) AS Quantity,
      Product_Cost,
      YEAR(Order_Date) as Year,
      QUARTER(Order_Date) as Quarter,
      CAST(
      CASE
        WHEN QUARTER(Order_Date) = 1
          THEN CONCAT(YEAR(Order_Date), '-01-01')
        WHEN QUARTER(Order_Date) = 2
          THEN CONCAT(YEAR(Order_Date), '-04-01')
        WHEN QUARTER(Order_Date) = 3
          THEN CONCAT(YEAR(Order_Date), '-07-01')
        WHEN QUARTER(Order_Date) = 4
          THEN CONCAT(YEAR(Order_Date), '-10-01')
      END AS DATE) AS Purchase_Date
      FROM Sales_df
      GROUP BY Vendor, Product,Product_Cost, YEAR(Order_Date), Quarter(Order_Date)
      ORDER BY Purchase_Date
      '''

    df_PO = ddb.sql(qry_PO).df()

    PO_rows = []
    for index, row in df_PO.iterrows():
      product = row['Product']
      Quantity_Sold = row['Quantity']
      Product_Cost = row['Product_Cost']
      Vendor = row['Vendor']
      Purchase_Date = row['Purchase_Date']
      Beg_Inventory = Master_Inventory.ReadInventory(product)

      New_Inventory = Beg_Inventory - Quantity_Sold
      if New_Inventory <= 0:
        Purchase_Quantity  = Quantity_Sold * 1.05
        New_Inventory = Beg_Inventory + Quantity_Sold * .05
        # Update Master Inventory and Record PO
        Master_Inventory.UpdateInventory(product, New_Inventory)
        PO_row = [product, Quantity_Sold, Quantity_Sold*Product_Cost, Purchase_Date,Vendor, Beg_Inventory, New_Inventory]
        PO_rows.append(PO_row)
      else:
       Master_Inventory.UpdateInventory(product, New_Inventory)

    self.PO_df = pd.DataFrame(PO_rows, columns=['Product', 'Quantity_Sold', 'Cost', 'Purchase_Date', 'Vendor', 'Beg_Inventory', 'New_Inventory'])

    cleaned_df = self.PO_df[['Purchase_Date', 'Cost']].copy()
    cleaned_df.rename(columns={'Purchase_Date': 'Date', 'Cost': 'Amount'}, inplace=True)
    self.RF.JournalEntries_Bulk(cleaned_df,1010,2000)
    # 1010 - Inventory | 2000 - Accounts Payable (To be Paid in 30 days) | 1000 - Cash
    cleaned_df['Date'] += pd.Timedelta(days=30) # Date Payed
    self.RF.JournalEntries_Bulk(cleaned_df,2000,1000)

    return [self.year,self.PO_df]
    

In [12]:

class RecordingFunctions:
  def __init__(self, Data_Manager):
    self.Data = Data_Manager

  # Bulk Data Processing
  def JournalEntries_Bulk(self, Clean_df, DebitAcc, CreditAcc, probability=None, val1=0, val2=0,p1=0,p2=0 ):
    if probability == None:
      pass
    elif probability.lower() == "dr":
      DebitAcc = random.choices([val1,val2],weights=[p1,p2],k=len(Clean_df))
    elif probability.lower() == "cr":
      CreditAcc = random.choices([val1,val2],weights=[p1,p2],k=len(Clean_df))

    df_debits = Clean_df[['Date','Amount']].copy()
    df_debits['Account'] = DebitAcc
    df_debits['Balance'] = "Debit"
    self.Data.ConcatData('journal',df_debits)

    df_credits = Clean_df[['Date','Amount']].copy()
    df_credits['Account'] = CreditAcc
    df_credits['Balance'] = "Credit"
    self.Data.ConcatData('journal',df_credits)



  # Row by Row Data Processing
  def JournalEntries_Single(self, Date, Amount, DebitAcc, CreditAcc):
    self.General_Journal = pd.DataFrame({"Date":[],"Amount":[],"Account":[],"Balance":[]})

    if isinstance(Amount, list) and isinstance(DebitAcc, list) and isinstance(CreditAcc, list):
      if len(Amount) == len(DebitAcc) + len(CreditAcc):
        len_ofDr = len(DebitAcc)
        DrAmounts = Amount[:len_ofDr]
        CrAmounts = Amount[len_ofDr:]
        Debits = zip(DebitAcc,DrAmounts)
        Credits = zip(CreditAcc,CrAmounts)
        for dr, amount in Debits:
          Journal_Entry = [Date, dr, amount, "Debit"]
          self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
        for cr, amount in Credits:
          Journal_Entry = [Date, cr, amount, "Credit"]
          self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
      else:
        print("ERROR Creating Journal Entries - Incorrect Number of Amounts")

    elif isinstance(Amount, int) or isinstance(Amount, float):
        Journal_Entry = [Date,DebitAcc, Amount, "Debit"]
        self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry

        Journal_Entry = [Date, CreditAcc, Amount, "Credit"]
        self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry

    else:
      print("ERROR Creating Journal Entry - Incorrect DataTypes")

    Data_manager.ConcatData('journal',self.General_Journal)

class Inventory_manager:
  def __init__(self,Inv=None):
    self.MasterInventory = {} if Inv == None else Inv

  def ReadInventory(self,product):
    return self.MasterInventory[product]

  def UpdateInventory(self,product,New_Inv):
    self.MasterInventory[product] = New_Inv


class Data_manager:
  def __init__(self, GJ=None, IS=None, BS=None, CF=None):
    self.MasterGeneralJournal = pd.DataFrame(columns = ["Date","Account","Amount","Balance"]) if GJ == None else GJ
    # Add the Rest of the Finacial Statements Here

  def RequestData(self,file):
    if 'journal' in file.lower():
      return self.MasterGeneralJournal
    elif 'income' in file.lower():
      pass
    elif 'balance' in file.lower():
      pass
    print(self.MasterGeneralJournal.info())

  def ConcatData(self,file,new_data):
    if 'journal' in file.lower():
      self.MasterGeneralJournal = pd.concat([new_data, self.MasterGeneralJournal],ignore_index=True)
    elif 'income' in file.lower():
      pass
    elif 'balance' in file.lower():
      pass
    print(self.MasterGeneralJournal.info())

  def PrintData(self,file):
    pass



In [15]:
Yr_Product_dict = {}
General_Journal = pd.DataFrame(columns = ["Date","Account","Amount","Balance"])

File_Manager = Data_manager()
Recording_Functions = RecordingFunctions(File_Manager)

for yr in rev_df["Year"]:
  # Processing Products
  Year_Rev = rev_df[rev_df['Year']==yr].values.tolist()[0]
  Processed_Products = PreProcessing(product_df, yr, Year_Rev).ProductProcessing()
  yr_product_data = Processed_Products[1]
  Inventory_data = Processed_Products[2] # New Function will manage Inventory Level State
  Inv_Manager = Inventory_manager(Inventory_data)

  for product in yr_product_data['yr_Product'].unique():
    CompanyGenerator = CompanyGeneration(Recording_Functions,yr, product, yr_product_data)
    CompanyGenerator.GenerateSales()
    CompanyGenerator.GeneratePO(Inv_Manager)
    break



  break


2010
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20601 entries, 0 to 20600
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     20601 non-null  datetime64[ns]
 1   Amount   20601 non-null  float64       
 2   Account  20601 non-null  object        
 3   Balance  20601 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 643.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41202 entries, 0 to 41201
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     41202 non-null  datetime64[ns]
 1   Amount   41202 non-null  float64       
 2   Account  41202 non-null  object        
 3   Balance  41202 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.3+ MB
None
test
Working on Inventory-----------------------
<class 'pandas.core.frame.DataFrame'

In [ ]:
# Testing New Journal Entry Data

cleandf = pd.DataFrame({"Date":[1,2,3,4,5,6,7,8,9,10],"Amount":[1,2,3,4,5,6,7,8,9,10]})

# RecordingFunctions().JournalEntries_Bulk(cleandf,1000,4000,"dr",1000,2000,40,60)

RecordingFunctions().JournalEntries_Single("1/1/2023", 100, 1000, 2000)

